In [ ]:
%%writefile in.nemd
# TODO: Understand this better and figure out the linear fit
# Sample LAMMPS input script for viscosity of FCC solid
# NEMD via fix deform and fix nvt/sllod

# Settings
variable	    x equal 10
variable	    y equal 10
variable        z equal 10

variable	    a equal 4.05
variable        T equal 2000
variable        P equal 0.0

variable	    srate equal 2.7 # velocity of top edge (Angstroms/ps)

# Problem setup
units		    metal
dimension	    3
boundary        p p p
atom_style	    atomic

lattice         fcc ${a}
region          simbox prism 0 $x 0 $y 0 $z 0.0 0.0 0.0 units lattice
create_box      1 simbox
create_atoms    1 box

pair_style      eam/fs
pair_coeff      * * Cu1.eam.fs Cu
mass            1  63.55

# 1. Equilibration run
velocity        all create $T 97287
fix             1 all npt temp $T $T $(100.0*dt) iso $P $P $(1000.0*dt)

thermo          1000
dump            equilibration all custom 50 dump.equilibration id type x y z vx vy vz 
run	            5000

undump         equilibration
unfix  	        1

# 2. Turn on NEMD shear and equilibrate some more
variable	    xyrate equal ${srate}/ly # Shear rate defined relative to perpendicular dimension (1/ps)

fix		        1 all nvt/sllod temp $T $T $(100.0*dt)

# Perform deformation every 1 timestep
# erate - engineering shear strain rate (1/time units)
fix		        2 all deform 1 xy erate ${xyrate} remap v

# The average in each chunk is computed every 5000 timesteps using 250 samples taken at intervals of 
# 20 timesteps from the preceding timesteps.
# For this case, there seems to be no difference between lower, center, and upper options.
compute         layers_center all chunk/atom bin/1d y center 0.05 units reduced
fix		        4 all ave/chunk 20 250 5000 layers_center vx file profile_center.nemd

compute		    usual all temp
compute		    tilt all temp/deform

thermo          1000
thermo_style	custom step temp c_usual epair etotal press pxy
thermo_modify	temp tilt

dump            nemd_equilibration all custom 50 dump.nemd_equilibration id type x y z vx vy vz
run		        50000
undump          nemd_equilibration

# 3. Data gathering run
# This only takes into account the top and bottom velocity points to fit the linear profile
# It doesn't even use fix 4, which is the only one that actually computes the profile
# The viscosity is calculated from the slope of the linear profile
variable        timestep equal step
variable	    visc equal -pxy/(v_srate/ly)

# The average in each chunk is computed every 1000 timesteps using 100 samples taken at intervals of 
# 10 timesteps from the preceding timesteps.
# The running average sums the average the average values computed every 1000 timesteps
fix             vprint all print 10 "${timestep} ${visc}" file viscosity_output.txt screen no
fix		        vave all ave/time 10 100 1000 v_visc ave running start 56000 file viscosity_averages.txt

thermo          1000
thermo_style	custom step temp etotal press pxy ly v_visc f_vave
thermo_modify	temp tilt

dump	        data_gathering all custom 50 dump.data_gathering id type x y z vx vy vz

run		        50000

Overwriting in.nemd


In [4]:
100000000*(1e-12)*20000

1.9999999999999998

In [19]:
import plotly.graph_objects as go

# Function to vx for a specific timestep using Plotly
def plot_vx(timestep):
    if timestep in chunk_data:
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=chunk_data[timestep]['vx'],
            y=chunk_data[timestep]['coord1'],
            mode='lines+markers',
            name=f'Timestep {timestep}'
        ))
        fig.update_layout(
            title=f'Velocity Profile at Timestep {timestep}',
            yaxis_title='y/l<sub>y</sub>',
            xaxis_title='vx',
            template='plotly_white',
            width=800,
            height=600 
        )
        fig.show()
    else:
        print(f"No data available for timestep {timestep}")

In [9]:
velocity_profile_file = 'profile_center.nemd'

# Initialize dictionaries to store the data for each chunk
chunk_data = {}

# Read the file and parse the data
with open(velocity_profile_file, 'r') as file:
    lines = file.readlines()
    for line in lines:
        if line.startswith('#') or line.strip() == '':
            continue
        parts = line.split()
        if len(parts) == 3:
            # This is a timestep line
            current_timestep = int(parts[0])
        elif len(parts) == 4:
            # This is a data line
            chunk = int(parts[0])
            coord = float(parts[1])
            ncount = float(parts[2])
            vx = float(parts[3])
            
            if current_timestep not in chunk_data:
                chunk_data[current_timestep] = {'chunk': [], 'coord1': [], 'ncounts': [], 'vx': []}
                
            chunk_data[current_timestep]['chunk'].append(chunk)
            chunk_data[current_timestep]['coord1'].append(coord)
            chunk_data[current_timestep]['ncounts'].append(ncount)
            chunk_data[current_timestep]['vx'].append(vx)

In [21]:
chunk_data

{10000: {'chunk': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,
   17,
   18,
   19,
   20],
  'coord1': [0.025,
   0.075,
   0.125,
   0.175,
   0.225,
   0.275,
   0.325,
   0.375,
   0.425,
   0.475,
   0.525,
   0.575,
   0.625,
   0.675,
   0.725,
   0.775,
   0.825,
   0.875,
   0.925,
   0.975],
  'ncounts': [199.832,
   199.48,
   201.564,
   203.088,
   199.168,
   197.976,
   201.4,
   199.412,
   199.352,
   202.26,
   200.336,
   199.688,
   200.42,
   201.708,
   199.28,
   198.268,
   198.452,
   200.132,
   200.152,
   198.032],
  'vx': [-0.913869,
   -0.785591,
   -0.786848,
   -0.772768,
   -0.554014,
   -0.369258,
   -0.241298,
   -0.186758,
   -0.0773374,
   -0.0325723,
   0.04115,
   0.110086,
   0.146354,
   0.191579,
   0.354127,
   0.630744,
   0.765387,
   0.95109,
   1.10601,
   1.34113]},
 15000: {'chunk': [1,
   2,
   3,
   4,
   5,
   6,
   7,
   8,
   9,
   10,
   11,
   12,
   13,
   14,
   15,
   16,


In [20]:
plot_vx(105000)

## Understand viscosity averaging

In [11]:
import numpy as np

# Read viscosity_output.txt into numpy array ignoring the first line
data = np.loadtxt('viscosity_output.txt', skiprows=1)

# Reshape the second column to group every 100 values
second_column = data[:, 1][1:]
reshaped = second_column[:].reshape(-1, 100)

# Compute the mean for each group
calculated_averages = reshaped.mean(axis=1)

# Compute the running average
calculated_running_averages = np.cumsum(calculated_averages) / np.arange(1, len(calculated_averages) + 1)

In [12]:
# Read the viscosity_averages.txt file into a numpy array ignoring the first two lines
averages = np.loadtxt('viscosity_averages.txt', skiprows=2)
running_averages = averages[:, 1]

In [13]:
# Compare the LAMMPS calculated running average with the one we computed
np.allclose(running_averages, calculated_running_averages, atol=1e-6)

True

In [14]:
running_averages

array([0.927771, 0.931235, 0.91869 , 0.912759, 0.918471, 0.914393,
       0.915803, 0.910069, 0.913466, 0.918714, 0.923954, 0.922469,
       0.925284, 0.921224, 0.919923, 0.919031, 0.920136, 0.920434,
       0.919982, 0.919444, 0.919022, 0.921352, 0.922024, 0.921346,
       0.921633, 0.923216, 0.922068, 0.921936, 0.922762, 0.922644,
       0.924842, 0.924176, 0.922896, 0.922776, 0.92258 , 0.925896,
       0.926938, 0.927029, 0.925717, 0.924952, 0.925183, 0.924124,
       0.92546 , 0.925798, 0.924781, 0.925197, 0.924153, 0.922969,
       0.923294, 0.923394])